In [ ]:
import os
import openpyxl
from pathlib import Path

UsePythonTypeIndex = False # A1을 (1,1)로 보고 싶을 때. (0,0)으로 보고 싶으면 True 입력

def openfile(filepath, filename, ValueOnly = False): # ValueOnly가 True이면 수식이 아니라 값으로 받아옴
    file = openpyxl.load_workbook('{0}/{1}.xlsx'.format(filepath, filename), data_only=ValueOnly)
    print("Log : open file '{0}' at '{1}'".format(filename, filepath))
    return file

def create(*WorksheetNames):
    Workbook = openpyxl.Workbook()
    for sheet in Workbook.sheetnames:
        Workbook.remove(Workbook[sheet]) # 디폴트 내용이 있으면 지우기
    Worksheets = []
    for WorksheetName in WorksheetNames:
        Worksheets.append(Workbook.create_sheet(title=WorksheetName, index=0)) # 시트 생성
        print("Log : excel sheet is created with title '{0}'".format(WorksheetName))
    output = (Workbook, *Worksheets)
    return output

def cell(Worksheet, col, row, *val):
    # 셀 값 접근, 예를 들어 cell(Sheet, 3, 2)은 Sheet에서 C2값을 나타냄
    if len(val) == 0:
        if UsePythonTypeIndex:
            return Worksheet.cell(column = col+1, row = row+1).value
        else:
            return Worksheet.cell(column = col, row = row).value
    # 셀 쓰기, 예를 들어 cell(Sheet, 3, 2, 'hello')는 Sheet에서 C2에 'hello'를 씀
    else:
        if UsePythonTypeIndex:
            Worksheet.cell(column = col + 1, row = row + 1, value = val[0])
        else:
            Worksheet.cell(column = col, row = row, value = val[0])

def save(filepath, filename, Workbook): # save(./resources, test, Ws)라고 하면, Ws가 ./resources/test.xlsx로 저장됨
    if not os.path.exists(filepath):
        os.makedirs(filepath)
    Workbook.save('{0}/{1}.xlsx'.format(filepath, filename))
    Workbook.close()
    print("Log : '{0}.xlsx' is saved at '{1}' folder".format(filename, filepath))

# 열고 읽는게 귀찮을 때 자동으로 열어서 데이터를 불러와주는 함수
def loadData(filepath, filename, *WorksheetNames, ValueOnly = False):
    # 출력형은 [{key1 : val1, key2 : val2, ...}, {key1 : val1, key2 : val2, ...}, ...] 형태로 구성
    Workbook = openfile(filepath, filename, ValueOnly = ValueOnly)
    dictionaryLists = []
    for WorksheetName in WorksheetNames:
        Worksheet = Workbook[WorksheetName]
        dictionaryList = []
        keyList = []
        for col in range(Worksheet.max_column):
            keyList.append(cell(Worksheet, col+1-UsePythonTypeIndex, 1-UsePythonTypeIndex))
        for row in range(Worksheet.max_row-1):
            dictionary = {}
            for col in range(Worksheet.max_column):
                if cell(Worksheet, col+1-UsePythonTypeIndex, row+1-UsePythonTypeIndex+1):
                    dictionary[keyList[col]] = cell(Worksheet, col+1-UsePythonTypeIndex, row+1-UsePythonTypeIndex+1)
                elif cell(Worksheet, col+1-UsePythonTypeIndex, row+1-UsePythonTypeIndex+1) == 0:
                    dictionary[keyList[col]] = cell(Worksheet, col + 1-UsePythonTypeIndex, row + 1-UsePythonTypeIndex+1)
            dictionaryList.append(dictionary)
        dictionaryLists.append(dictionaryList)
    print('Log : data loaded')
    if len(WorksheetNames) == 1:
        return dictionaryList
    else:
        return tuple(dictionaryLists)

# 생성하고 쓰는게 귀찮을 때 자동으로 만들어서 적고 저장해주는 함수
def writeData(filepath, filename, WorksheetName, dictionaryList, keyList = []):
    # DictionaryList는 [{key1 : val1, key2 : val2, ...}, {key1 : val1, key2 : val2, ...}, ...] 형태로 구성
    # KeyList는 Key 순서를 강제하고 싶을 때 [key1, key, ...] 방식으로 구성해서 넣기, 디폴트는로는 안넣어도 됨
    if Path('{0}/{1}.xlsx'.format(filepath, filename)).is_file():
        Workbook = openfile(filepath, filename)
        if WorksheetName in Workbook.sheetnames:
            Worksheet = Workbook[WorksheetName]
        else:
            Worksheet = Workbook.create_sheet(WorksheetName)
    else:
        Workbook, Worksheet = create(WorksheetName)
    if len(keyList) == 0:
        for key in dictionaryList[0].keys():
            keyList.append(key)
    col = 0
    for key in keyList:
        cell(Worksheet, col+1-UsePythonTypeIndex, 1-UsePythonTypeIndex, key)
        for row in range(len(dictionaryList)):
            if key in dictionaryList[row]:
                cell(Worksheet, col+1-UsePythonTypeIndex, row+1-UsePythonTypeIndex+1, dictionaryList[row][key])
        col += 1
    print('Log : data written')
    save(filepath, filename, Workbook)

# 시트에서 키의 위치를 찾아주는 함수 : A3에 있는 키일 경우 3을 출력, B3에 있으면 키로 보지 않음
def findKeyIndex(Worksheet, Key):
    for col in range(Worksheet.max_column):
        if cell(Worksheet, col+1-UsePythonTypeIndex, 1-UsePythonTypeIndex) == Key:
            return col+1-UsePythonTypeIndex
    return -1

# 주어진 키에 해당하는 데이터 리스트를 불러오는 함수
def getListFromKey(Worksheet, Key, IsNumber = True):
    KeyIndex = findKeyIndex(Worksheet, Key)
    List = []
    for row in range(Worksheet.max_row-1):
        if cell(Worksheet, KeyIndex, row+1-UsePythonTypeIndex+1):
            if IsNumber:
                List.append(float(str(cell(Worksheet, KeyIndex, row+1-UsePythonTypeIndex+1)).replace(",", "")))
            else:
                List.append(cell(Worksheet, KeyIndex, row + 1 - UsePythonTypeIndex + 1))
    return List

# 마지막줄을 찾아서 그 아래에 리스트값을 입력하는 함수
def writeList(Worksheet, list, is_first=False):
    row = Worksheet.max_row
    if is_first: row = 0
    for i, e in enumerate(list):
        cell(Worksheet, i+1, row+1, e)

#####샘플 사용 코드#####
def run():
    filepath = '../resources'  # 파일 패스 설정. .은 py 파일이 있는 폴더를 의미
    filename = 'sample'  # 파일 이름 설정. .xlsx는 빼고 써야 함
    Workbook = openfile(filepath, filename)  # 파일을 연다
    Worksheet1 = Workbook['Sheet1']  # 시트를 연다
    print(cell(Worksheet1, 1, 1))  # 시트 안에 있는 A1 값 출력
    cell(Worksheet1, 1, 1, 'hello')  # A1값을 'hello'로 바꾸기
    print(getListFromKey(Worksheet1, 'hello'))  # 'hello'라는 키에 해당하는 데이터 리스트 불러오기 : [A2, A3, ...]에 해당
    # save(filepath, 'sample2', Workbook) # 다른 이름으로 저장하기
    save(filepath, filename, Workbook)  # 다시 저장하고 닫기
    dict = loadData(filepath, filename, 'Sheet1')  # 파일을 열어 딕셔너리 배열로 변환
    print(dict)  # 배열 출력
    writeData(filepath, 'sample_copied', 'Sheet1_copied', dict)  # 딕셔너리 배열 복사본으로 저장

if __name__ == "__main__":
    run()